# Log Regression

In this document, we investigate log regression models. 

In [17]:

import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold
from sklearn.feature_selection import f_classif
import itertools
import sys
import importlib
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
# from sklearn.neighbors import KNeighborsClassifier, NeighborhoodComponentsAnalysis 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, make_scorer
from joblib import Parallel, delayed, parallel_backend
from threadpoolctl import threadpool_limits
import matplotlib.pylab as plt
import os
from sklearn.inspection import permutation_importance
from scipy.stats import norm, t
import warnings

sys.path.append("../")
from proj_mod import training
importlib.reload(training);

## Data importing 

In [18]:
df=pd.read_csv("../data/raw.csv")
features=list(df.columns)[1:]
target=["Y"]
feat=df[features]
tar=df[target]
# x_t, x_v, y_t, y_v= train_test_split(feat,tar, test_size=0.2, random_state=0, stratify=tar["Y"])
n_splits=5

## For all raw features 

Again, we use anova f test to determine a list of features that we will use. 

In [19]:
eva_pipe=Pipeline([("DataCreater", training.data_creator()),("DataSelector",training.data_selector())])
tar_arr=np.ravel(tar.values)
eva_pipe.fit(X=feat,y=tar)
eva_out=eva_pipe.transform(X=feat)
eva_out.columns

Index(['X1', 'X3', 'X5', 'X6', 'mean', 'F_w_mean', 'above_4', 'above_5'], dtype='object')

In [20]:
eva_pipe["DataSelector"].sel_

,features,f score,p value,X1,X2,X3,X4,X5,X6,mean,F_w_mean,above_3,above_4,above_5,Y
0,X1,10.561708,0.001486,1.000000,0.059797,0.283358,0.087541,0.432772,0.411873,0.607460,0.834641,0.266199,0.492355,0.604855,0.280160
2,X3,2.886959,0.091807,0.283358,0.184129,1.000000,0.302618,0.358397,0.203750,0.676149,0.532371,0.442280,0.638649,0.481689,0.150838
4,X5,6.582716,0.011488,0.432772,0.039996,0.358397,0.293115,1.000000,0.320195,0.712786,0.806779,0.491804,0.616787,0.586695,0.224522
5,X6,3.586849,0.060568,0.411873,-0.062205,0.203750,0.215888,0.320195,1.000000,0.540096,0.574523,0.261704,0.458477,0.490605,0.167669
6,mean,7.306094,0.007836,0.607460,0.426097,0.676149,0.557803,0.712786,0.540096,1.000000,0.869373,0.687659,0.848710,0.773920,0.235885
7,F_w_mean,12.615311,0.000542,0.834641,0.078909,0.532371,0.298662,0.806779,0.574523,0.869373,1.000000,0.498831,0.743851,0.761327,0.303878
9,above_4,7.194813,0.008308,0.492355,0.268810,0.638649,0.521454,0.616787,0.458477,0.848710,0.743851,0.465645,1.000000,0.546995,0.234181
10,above_5,6.520675,0.011874,0.604855,0.215269,0.481689,0.389949,0.586695,0.490605,0.773920,0.761327,0.229256,0.546995,1.000000,0.223515


The penalties should be able to replace exhaust feature subset search for removing high colinearity. But it does not heart to try. 

In [21]:
penalty_choice=["l1","l2","elasticnet", None]
range_feat_combin = training.all_combin(eva_out.columns)

n_split = 5
n_repeats = 20

RSKF = RepeatedStratifiedKFold(n_splits=n_split, random_state=420, n_repeats=n_repeats)

splits = list(RSKF.split(X=feat, y=tar))

def evaluate_combo(list_f_sel_tuple, penalty, splits, feat, tar):
    """
    Evaluate one (feature_set, n_neighbors) across all CV folds.
    
    :param list_f_sel_tuple: A tuple indicating a combination. 
    :param penalty: The penalty of logregression used. 
    :param splits: A list of the pre generated splits. 
    :param feat: The feat df. 
    :param tar: The tar df. 
    :return: A dict with all the stats we want. 
    """
    list_f_sel = list(list_f_sel_tuple) 
    fold_acc = []
    fold_f1 = []

    for train_index, test_index in splits:
        x_tr, x_te = feat.iloc[train_index], feat.iloc[test_index]
        y_tr, y_te = tar.iloc[train_index], tar.iloc[test_index]

        y_tr = np.ravel(y_tr.values)
        y_te = np.ravel(y_te.values)
        
    
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')

            pipe = Pipeline([
                ("DataCreate", training.data_creator()),
                ("DataSelect", training.data_selector(force=list_f_sel)),
                ("scale", StandardScaler()),
                ("LogReg", LogisticRegression(penalty=penalty,solver="saga", l1_ratio=0.1)),
            ])
            
            pipe.fit(X=x_tr, y=y_tr)
            y_p = pipe.predict(X=x_te)

        fold_acc.append(accuracy_score(y_true=y_te, y_pred=y_p))
        fold_f1.append(f1_score(y_true=y_te, y_pred=y_p))

    str_features = ",".join(list_f_sel)
    acc_mean = float(np.mean(fold_acc))
    acc_std  = float(np.std(fold_acc))
    f1_mean  = float(np.mean(fold_f1))
    f1_std   = float(np.std(fold_f1))
    above_73 = float((np.array(fold_acc) >= 0.73).sum() / (len(splits)))
    norm_above_73 = float(1-norm.cdf(0.73, loc=acc_mean, scale=acc_std))
    acc_mean_above_73 = float(1-norm.cdf(0.73, loc=acc_mean, scale=acc_std/np.sqrt(len(splits))))

    msg = (
        "_"*20 + "\n"
        + f"Currently used features {str_features} and penalty {penalty}.\n"
        + f"This combo has f1 mean {f1_mean} and f1 std {f1_std}, \n"
        + f"with acc mean {acc_mean} acc std {acc_std}, "
        + f"and sureness of beating 73% {above_73}.\n"
        + "_"*20
    )

    return {
        #Hyper-parameters
        "features": str_features,
        "penalty": penalty,
        #Performance
        "acc_mean": acc_mean,
        "acc_std": acc_std,
        "f1_mean": f1_mean,
        "f1_std": f1_std,
        "above_73": above_73,
        "norm_above_73": norm_above_73,
        "acc_mean_above_73": acc_mean_above_73,
        #Log
        "log": msg,
    }

jobs = list(itertools.product(range_feat_combin, penalty_choice))

results = Parallel(n_jobs=-1, backend="loky", verbose=10)(
    delayed(evaluate_combo)(feat_sel, penalty, splits, feat, tar)
    for feat_sel, penalty in jobs
)

list_feat      = [r["features"] for r in results]
list_penalty        = [r["penalty"] for r in results]
list_acc_mean  = [r["acc_mean"] for r in results]
list_acc_std   = [r["acc_std"] for r in results]
list_f1_mean   = [r["f1_mean"] for r in results]
list_f1_std    = [r["f1_std"] for r in results]
list_above_73  = [r["above_73"] for r in results]
list_norm_above_73 = [r["norm_above_73"] for r in results] 
list_acc_mean_above_73 = [r["acc_mean_above_73"] for r in results]


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.


[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   16.2s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:   16.9s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:   24.2s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:   31.4s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   38.8s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   45.9s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:   48.7s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 256 tasks      | elapsed:  2.1min
[Paralle

In [22]:
df_results = pd.DataFrame({
    #Hyper-parameters
    "features": list_feat,
    "penalty": list_penalty,
    #Performances
    "acc_mean": list_acc_mean,
    "acc_std": list_acc_std,
    "f1_mean": list_f1_mean,
    "f1_std": list_f1_std,
    "above_73": list_above_73,
    "norm_above_73": list_norm_above_73, 
    "acc_mean_above_73": list_acc_mean_above_73
})

In [23]:
df_results.sort_values(by=["above_73"],ascending=False)

,features,penalty,acc_mean,acc_std,f1_mean,f1_std,above_73,norm_above_73,acc_mean_above_73
3,X1,None,0.635523,0.077083,0.663823,0.081786,0.10,0.110166,0.0
2,X1,elasticnet,0.632354,0.073982,0.664249,0.081117,0.08,0.093440,0.0
1,X1,l2,0.632354,0.073982,0.664249,0.081117,0.08,0.093440,0.0
26,above_4,elasticnet,0.610508,0.083655,0.680617,0.106366,0.05,0.076589,0.0
546,"X3,X5,above_4,above_5",elasticnet,0.556523,0.086685,0.616065,0.094908,0.05,0.022684,0.0
...,...,...,...,...,...,...,...,...,...
990,"X1,X3,X5,X6,mean,F_w_mean,above_5",elasticnet,0.543662,0.067215,0.609816,0.071497,0.00,0.002783,0.0
989,"X1,X3,X5,X6,mean,F_w_mean,above_5",l2,0.545631,0.066618,0.611738,0.071695,0.00,0.002824,0.0
7,X3,None,0.552369,0.072944,0.675819,0.058102,0.00,0.007443,0.0
6,X3,elasticnet,0.550785,0.071364,0.675156,0.057392,0.00,0.006015,0.0


In [25]:
df_results.sort_values(by=["f1_mean"],ascending=False)

,features,penalty,acc_mean,acc_std,f1_mean,f1_std,above_73,norm_above_73,acc_mean_above_73
12,X6,l1,0.605154,0.066580,0.709280,0.065599,0.02,0.030387,0.0
14,X6,elasticnet,0.603954,0.067969,0.705747,0.072236,0.02,0.031837,0.0
13,X6,l2,0.603954,0.067969,0.705747,0.072236,0.02,0.031837,0.0
15,X6,None,0.605569,0.071387,0.704744,0.078659,0.03,0.040663,0.0
24,above_4,l1,0.617708,0.079668,0.690989,0.099157,0.05,0.079343,0.0
...,...,...,...,...,...,...,...,...,...
337,"X6,mean,above_5",l2,0.526462,0.088957,0.576594,0.097760,0.00,0.011067,0.0
557,"X3,X6,mean,above_5",l2,0.523262,0.086536,0.573786,0.094723,0.00,0.008446,0.0
339,"X6,mean,above_5",None,0.522508,0.086066,0.573583,0.095267,0.00,0.007958,0.0
558,"X3,X6,mean,above_5",elasticnet,0.522062,0.086666,0.573154,0.094703,0.00,0.008213,0.0


I would still prefer to consider 3 as the best model. It only used X1 and no penalty. 

In [26]:
df_results.to_csv("../data/LogReg_results_exhaust_raw6.csv", index=False)

Log Regression is not really performing here, and is not really giving me anything too interesting: It is literally saying we are better off just stare at X1 only. I do not think further investigation will be interesting. 